In [1]:
# Tensorflow 1.6.0 version code 
# This code provide the basic cnn
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

C:\Users\HCILab\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
            self.is_training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            L1 = tf.layers.conv2d(X_img, 32, [3, 3],
                                 padding='SAME', activation=tf.nn.relu)
            L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2], padding='SAME')
            L1 = tf.layers.dropout(L1, self.keep_prob, self.is_training)
            
            L2 = tf.layers.conv2d(L1, 64, [3, 3],
                                 padding='SAME', activation=tf.nn.relu)
            L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2], padding='SAME')
            L2 = tf.layers.dropout(L2, self.keep_prob, self.is_training)
            
            L3 = tf.layers.conv2d(L2, 128, [3, 3],
                                 padding='SAME', activation=tf.nn.relu)
            L3 = tf.layers.max_pooling2d(L3, [2, 2], [2, 2], padding='SAME')
            L3 = tf.layers.dropout(L3, self.keep_prob, self.is_training)
            L3_flat = tf.layers.flatten(L3)
            
            L4 = tf.layers.dense(L3_flat, 625, activation=tf.nn.relu)
            L4 = tf.layers.dropout(L4, self.keep_prob, self.is_training)
            
            self.logits = tf.layers.dense(L4, 10, 
                                 bias_initializer=tf.contrib.layers.xavier_initializer())
            
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self.logits, labels= self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def predict(self, x_test, keep_prob=1.0):
        return self.sess.run(self.logits, feed_dict={self.X:x_test, self.keep_prob: keep_prob})
        
    def get_acccuracy(self, x_test, y_test, keep_prob=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y:y_test,
                                                       self.keep_prob:keep_prob})
        
    def train(self, x_data, y_data, keep_prop=0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})

In [3]:

# initialize
sess = tf.Session()
m1 = Model(sess, 'm1')

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))


Learning Started!
Epoch: 0001 cost = 0.164430845
Epoch: 0002 cost = 0.042854028
Epoch: 0003 cost = 0.030892440
Epoch: 0004 cost = 0.022325219
Epoch: 0005 cost = 0.017607915
Epoch: 0006 cost = 0.014791263
Epoch: 0007 cost = 0.011237337
Epoch: 0008 cost = 0.011090888
Epoch: 0009 cost = 0.008227763
Epoch: 0010 cost = 0.009701179
Epoch: 0011 cost = 0.007570400
Epoch: 0012 cost = 0.006677476
Epoch: 0013 cost = 0.004355405
Epoch: 0014 cost = 0.008095690
Epoch: 0015 cost = 0.005628462
Learning Finished!


AttributeError: 'Model' object has no attribute 'get_accuracy'